# Final Model

## Importing libraries and dataframes

In [1]:
# main
import pandas as pd
import numpy as np
import time

# spotify
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

## Setting everything for running the model
## Spotify API & Authentication

In [2]:
import config

In [3]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))


## Scaling and clustering the songs

In [4]:
spotify_playlist = pd.read_csv("../spotify_playlist.csv")
final_list = spotify_playlist.iloc[:, 2:]
final_list.head()

,duration,danceability,energy,acousticness,instrumentalness,liveness,loudness,speechiness,valence,tempo
0,369600,0.482,0.721,0.731000,0.000000,0.1890,-6.839,0.0321,0.557,95.263
1,301240,0.485,0.863,0.000012,0.016200,0.1380,-9.027,0.0495,0.767,116.835
2,337413,0.364,0.457,0.290000,0.000106,0.9220,-14.162,0.0675,0.175,163.219
3,219146,0.398,0.413,0.082200,0.000025,0.0891,-10.934,0.0388,0.331,133.574
4,160892,0.518,0.756,0.735000,0.000062,0.3170,-10.851,0.0915,0.968,166.429


In [5]:
# initilazing the scaler
scaler = StandardScaler()

# scaling the playlist
playlist_scaled = scaler.fit_transform(final_list)
playlist_scaled

array([[ 0.99246275, -0.50112909,  0.44606667, ..., -0.52005721,
        -0.08095904, -0.93184138],
       [ 0.41750491, -0.48313242,  1.05368433, ..., -0.28751635,
         0.77214694, -0.1602277 ],
       [ 0.72174643, -1.2089984 , -0.68358871, ..., -0.04695684,
        -1.63279944,  1.49889194],
       ...,
       [-0.57664395,  0.7706361 ,  0.39899769, ...,  3.59484688,
        -0.65782118, -0.0483058 ],
       [-0.247616  , -1.40096296,  0.78410748, ..., -0.51337501,
         1.03620356,  0.99887397],
       [-0.14085854, -0.39914792,  0.62578457, ..., -0.00820003,
         1.29213535,  1.12181279]])

## KMeans

The greater the Silouhette, the greater the separation between the clusters, in other words, the clusters will be more defined.

Looking into the results , the Elbow Method doesn't show us a clear winner for the n_clusters we should use. On the other hand, thanks to the Silhoutte Score we see that this score reach it's peak at 7. For that, we will redo the clustering with this new value in mind.


In [6]:
# intilize the model
kmeans = KMeans(n_clusters=7, random_state=42)

# fit the model
kmeans.fit(playlist_scaled)

KMeans(n_clusters=7, random_state=42)

## Running the Final Model
### Loading the CSV Files

In [7]:
billboard = pd.read_csv("../top_hot_100.csv")
playlist = pd.read_csv("../playlist_with_clusters.csv")

## Testing the Model

In [8]:
#User inputs a song. If it's on the DF, it will return another one, if not it will search on Spotify.
def input_song(df):
    choice = input("Welcome to the IronCarla Song Recommender. Write down a song to get a recommendation: \n").lower().strip()
    
    if choice in df.str.lower().values:
        return 1, choice # billboard
    else:
        return 0, choice # if not on billboard, spotify 

In [9]:
album, choice = input_song(billboard["album"])

# Choice from Billboard 100 songs 
if album == 1:
    suggestion = billboard.sample()
    suggestion_album = suggestion["album"].values.flatten()
    suggestion_artist = suggestion["artist"].values.flatten()
    suggestion_album = str(suggestion_album).lstrip("[").rstrip("]")
    suggestion_artist = str(suggestion_artist).lstrip("[").rstrip("]")

    time.sleep(1.5)
    print("\nLoading..")
    time.sleep(1.5)
    print("Loading...\n")
    time.sleep(0.5)
    print("We found something!\n")
    print("This song is HOT! Wait to get another hot song recommended...")
    print(f"Here's our recommedation: {suggestion_album} from {suggestion_artist}. What a hit!")
    print("\nThanks for using IronCarla Song Recommender")

# Choice not from Billboard 100 songs, hence search based on 10k songs playlist
elif album == 0:
    spoti_song = sp.search(q=choice)
        
    print(spoti_song["tracks"]["items"][0]["name"], " - ", spoti_song["tracks"]["items"][0]["artists"][0]["name"])
    song_features_spoti = pd.DataFrame(sp.audio_features(spoti_song["tracks"]["items"][0]["uri"]))
    spoti_features = song_features_spoti[["danceability", "energy", "mode", "acousticness", "instrumentalness", "liveness",
                          "loudness", "speechiness", "valence", "tempo"]]

    features_spoti_scaled = scaler.transform(spoti_features)
    spoti_cluster = kmeans.predict(features_spoti_scaled)
    match_song = playlist[playlist["cluster"] == int(spoti_cluster)]["song"]
    
    song_suggestion = match_song.sample()

    clean_song = list(song_suggestion)
    clean_song_2 = str(clean_song).lstrip("[").rstrip("]")    

    print("Wops, seems your song is not hot, but wait to get a recommendation based on the audio features of the song...\n")

    time.sleep(1.5)
    print("Loading..")
    time.sleep(1.5)
    print("Loading...\n")
    time.sleep(0.5)

    print("Finally! Here's your perfect song:\n")
    print(f"{clean_song_2}\n")
    
        
    # Continue recommending 
    while True:
        redo = input(f"Do you want to predict another song? y/n: ")
        if redo == "y":
            print("Here's another recommendation:\n")
            spoti_suggestion = match_song.sample()
            clean_suggestion = list(spoti_suggestion)
            clean_suggestion_2 = str(clean_suggestion).lstrip("[").rstrip("]")
            print(f"{clean_suggestion_2}\n")
            
        elif redo == "n":
            print("\nThanks for using IronCarla Song Recommender")
            break
        else:
            print("Sorry, we couldn't detect your choice. Remember to write 'y' or 'n'")

Welcome to the IronCarla Song Recommender. Write down a song to get a recommendation: 
Fly me to the moon
Fly Me To The Moon (In Other Words)  -  Frank Sinatra
Wops, seems your song is not hot, but wait to get a recommendation based on the audio features of the song...

Loading..
Loading...

Finally! Here's your perfect song:

'Shapes Of Things'

Do you want to predict another song? y/n: y
Here's another recommendation:

'The Sky Is Crying'

Do you want to predict another song? y/n: n

Thanks for using IronCarla Song Recommender


## Future Improvements to model
- include urls to recommendations to make the demo nicer 
- bold artist and song
- special characters